FastAPI with SQLite

In [8]:
from fastapi import FastAPI
import sqlite3
from contextlib import asynccontextmanager

async def lifespan(app: FastAPI):
    conn = sqlite3.connect("nutrition.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS foods (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            calories INTEGER NOT NULL
        )
    """)
    # Step 2: Insert 20 foods only if the table is empty
    cursor.execute("SELECT COUNT(*) FROM foods")
    if cursor.fetchone()[0] == 0:
        cursor.executemany("INSERT INTO foods (name, calories) VALUES (?, ?)", [
            ("Apple", 95), ("Banana", 105), ("Orange", 62), ("Mango", 99),
            ("Grapes", 67), ("Pineapple", 82), ("Strawberry", 33),
            ("Avocado", 160), ("Blueberry", 57), ("Kiwi", 42),
            ("Watermelon", 30), ("Pear", 101), ("Peach", 58),
            ("Plum", 46), ("Cherry", 50), ("Fig", 74),
            ("Guava", 68), ("Papaya", 59), ("Pomegranate", 83), ("Coconut", 354)
        ])
        conn.commit()
    conn.close()
    yield

# Step 3: Initialize FastAPI app
app = FastAPI(lifespan=lifespan)


In [9]:
@app.get("/")
def read_root():
    return {"message": "Hello World with SQLite!"}

# Step 5: /foods route to return all food items
@app.get("/foods")
def get_foods():
    conn = sqlite3.connect("nutrition.db")
    cursor = conn.cursor()
    cursor.execute("SELECT id, name, calories FROM foods")
    rows = cursor.fetchall()
    conn.close()
    return {"foods": [{"id": r[0], "name": r[1], "calories": r[2]} for r in rows]}